### Finviz Data Cleaning and Transformation Pipeline

This notebook processes the raw data downloaded from Finviz, cleans it, and transforms it into an analysis-ready format.

**Workflow:**

1.  **Load Data:** Reads a raw Finviz `.parquet` file for a specific date.
2.  **Feature Engineering:** Creates new composite columns (`Info`, `MktCap AUM`).
3.  **Data Type Conversion:**
    *   Converts currency strings (e.g., `1.5B`, `250K`) into numeric values in millions.
    *   Converts percentage strings (e.g., `12.5%`) into numeric values.
    *   Converts other object columns to their proper numeric types.
4.  **Final Processing:** Sorts the data by market capitalization, sets the `Ticker` as the index, and adds a `Rank` column.
5.  **Save & Verify:** Saves the cleaned DataFrame to a new `.parquet` file and verifies the saved file.

### Setup and Configuration

**This is the only cell you need to modify.** It contains all imports, paths, and lists of columns for processing.

In [1]:
import sys
from pathlib import Path
import pandas as pd
import numpy as np

# --- Project Path Setup ---
NOTEBOOK_DIR = Path.cwd()
ROOT_DIR = NOTEBOOK_DIR.parent if NOTEBOOK_DIR.name == 'notebooks' else NOTEBOOK_DIR
if str(ROOT_DIR) not in sys.path:
    sys.path.append(str(ROOT_DIR))

SRC_DIR = ROOT_DIR / 'src'
if str(SRC_DIR) not in sys.path:
    sys.path.append(str(SRC_DIR))

# Import config and custom utils now that path is set
from config import DATE_STR, DOWNLOAD_DIR, DEST_DIR
import utils

# --- File Path Configuration ---
# Build paths using pathlib for cross-platform compatibility
SOURCE_PATH = Path(DOWNLOAD_DIR) / f'df_finviz_{DATE_STR}_stocks_etfs.parquet'
DEST_PATH = Path(DEST_DIR) / f'{DATE_STR}_df_finviz_stocks_etfs.parquet'

# --- Column Processing Configuration ---
# Define which columns need specific cleaning operations.

# Columns to combine into the 'Info' column
INFO_COLS = ["Sector", "Industry", "Single Category", "Asset Type"]

# Columns with abbreviated currency values (B, M, K) to be converted to millions
CURRENCY_COLS = [
    'Market Cap', 'AUM', 'Sales', 'Income', 'Outstanding', 'Float', 
    'Short Interest', 'Avg Volume', 'Flows 1M', 'Flows 3M', 'Flows YTD',
    'MktCap AUM' # This is the new column we create
]

# Other columns that are numeric but stored as strings (objects)
# Note: Percentage columns are detected automatically in Step 3.
OTHER_NUMERIC_COLS = [
    "No.", "P/E", "Fwd P/E", "PEG", "P/S", "P/B", "P/C", "P/FCF",
    "Book/sh", "Cash/sh", "Dividend TTM", "EPS", "EPS next Q", "Short Ratio",
    "Curr R", "Quick R", "LTDebt/Eq", "Debt/Eq", "Beta", "ATR", "RSI",
    "Employees", "Recom", "Rel Volume", "Volume", "Target Price",
    "Prev Close", "Open", "High", "Low", "Price", "Holdings"
]

# --- Notebook Setup ---
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)
pd.set_option('display.width', 2500)
%load_ext autoreload
%autoreload 2

# --- Verification ---
print(f"Source file: {SOURCE_PATH}")
print(f"Destination file: {DEST_PATH}")
print(f"Processing for date: {DATE_STR}")

Source file: C:\Users\ping\Downloads\df_finviz_2025-06-26_stocks_etfs.parquet
Destination file: c:\Users\ping\Files_win10\python\py311\stocks_v0_works\data\2025-06-26_df_finviz_stocks_etfs.parquet
Processing for date: 2025-06-26


### Step 1: Load Raw Data

Load the source Parquet file into a pandas DataFrame.

In [2]:
print(f"--- Step 1: Loading data from {SOURCE_PATH.name} ---")

try:
    df = pd.read_parquet(SOURCE_PATH, engine='pyarrow')
    print("Data loaded successfully.")
    df.info()
    display(df.head(3))
except FileNotFoundError:
    print(f"ERROR: Source file not found at {SOURCE_PATH}")
    df = None  # Ensure df is None if loading fails
except Exception as e:
    print(f"An error occurred during file loading: {e}")
    df = None

--- Step 1: Loading data from df_finviz_2025-06-26_stocks_etfs.parquet ---
Data loaded successfully.
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1560 entries, 0 to 1559
Columns: 111 entries, No. to Tags
dtypes: object(111)
memory usage: 1.3+ MB


,No.,Ticker,Company,Index,Sector,Industry,Country,Exchange,Market Cap,P/E,Fwd P/E,PEG,P/S,P/B,P/C,P/FCF,Book/sh,Cash/sh,Dividend,Dividend TTM,Dividend Ex Date,Payout Ratio,EPS,EPS next Q,EPS This Y,EPS Next Y,EPS Past 5Y,EPS Next 5Y,Sales Past 5Y,Sales Q/Q,EPS Q/Q,EPS YoY TTM,Sales YoY TTM,Sales,Income,EPS Surprise,Revenue Surprise,Outstanding,Float,Float %,Insider Own,Insider Trans,Inst Own,Inst Trans,Short Float,Short Ratio,Short Interest,ROA,ROE,ROIC,Curr R,Quick R,LTDebt/Eq,Debt/Eq,Gross M,Oper M,Profit M,Perf Week,Perf Month,Perf Quart,Perf Half,Perf Year,Perf YTD,Beta,ATR,Volatility W,Volatility M,SMA20,SMA50,SMA200,50D High,50D Low,52W High,52W Low,52W Range,All-Time High,All-Time Low,RSI,Earnings,IPO Date,Optionable,Shortable,Employees,Change from Open,Gap,Recom,Avg Volume,Rel Volume,Volume,Target Price,Prev Close,Open,High,Low,Price,Change,Single Category,Asset Type,Expense,Holdings,AUM,Flows 1M,Flows% 1M,Flows 3M,Flows% 3M,Flows YTD,Flows% YTD,Return% 1Y,Return% 3Y,Return% 5Y,Tags
0,741,SOLV,Solventum Corp,S&P 500,Healthcare,Medical Instruments & Supplies,USA,NYSE,12.91B,34.39,12.55,-,1.55,3.96,24.18,33.02,18.85,3.09,-,-,-,0.00%,2.17,1.45,-16.86%,6.73%,-,-0.19%,-,2.68%,-42.89%,-70.83%,1.28%,8.31B,378.00M,10.64%,2.66%,173.01M,138.41M,80.00%,20.00%,0.00%,67.78%,5.05%,3.03%,3.58,4.20M,2.59%,10.63%,3.41%,1.19,0.85,2.40,2.53,55.45%,10.67%,4.55%,2.75%,2.90%,0.53%,11.67%,45.48%,12.97%,0.69,1.70,2.05%,2.04%,0.67%,5.01%,4.35%,-2.29%,18.25%,-13.14%,58.25%,47.16 - 85.92,-22.30%,58.25%,55.85,May 08/a,3/26/2024,Yes,Yes,22000,-0.39%,-0.04%,2.77,1.17M,0.53,"616,831",81.57,74.95,74.92,75.68,74.28,74.63,-0.43%,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
1,742,REG,Regency Centers Corporation,S&P 500,Real Estate,REIT - Retail,USA,NASD,12.88B,33.27,28.89,3.83,8.75,1.97,164.02,33.55,35.76,0.43,4.02%,2.78,6/11/2025,128.50%,2.12,0.55,8.25%,6.90%,8.16%,8.69%,6.21%,4.44%,1.46%,3.41%,7.39%,1.47B,386.55M,2.98%,3.56%,181.05M,180.20M,99.53%,0.73%,-8.52%,103.47%,0.69%,3.45%,5.50,6.21M,3.17%,5.83%,3.31%,0.97,0.97,0.74,0.79,43.40%,35.96%,26.24%,-0.25%,-2.22%,-3.14%,-4.42%,13.30%,-4.59%,1.01,1.18,1.51%,1.53%,-1.14%,-1.73%,-2.76%,-5.06%,1.73%,-9.77%,16.33%,60.64 - 78.18,-24.55%,366.38%,45.07,Apr 29/a,10/29/1993,Yes,Yes,500,0.54%,0.19%,1.70,1.13M,1.06,"1,198,269",78.89,70.03,70.16,70.60,69.95,70.54,0.73%,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-
2,743,PAA,Plains All American Pipeline LP,-,Energy,Oil & Gas Midstream,USA,NASD,12.84B,19.65,12.19,13.74,0.26,1.69,30.08,6.24,10.80,0.61,8.50%,1.40,5/1/2025,173.45%,0.93,0.35,7.18%,-7.41%,-22.68%,1.43%,8.56%,-1.01%,68.23%,-20.48%,3.40%,50.17B,653.00M,-10.51%,-15.05%,703.78M,457.18M,64.96%,35.00%,0.00%,39.16%,-5.71%,2.38%,2.95,10.87M,3.46%,9.43%,3.60%,1.01,0.94,0.88,0.93,3.96%,3.19%,1.30%,-0.81%,8.18%,-10.40%,7.79%,3.93%,6.91%,0.66,0.46,2.35%,2.37%,3.65%,5.15%,0.70%,-3.08%,14.88%,-13.05%,17.24%,15.57 - 21.00,-70.11%,508.67%,59.94,May 09/b,11/18/1998,Yes,Yes,4200,1.39%,0.33%,2.40,3.68M,0.58,"2,152,966",21.00,17.95,18.01,18.27,17.96,18.26,1.73%,-,-,-,-,-,-,-,-,-,-,-,-,-,-,-


### Step 2: Feature Engineering - Create Composite Columns

Combine existing columns to create more meaningful features: `Info` and `MktCap AUM`.

In [3]:
if df is not None:
    print("\n--- Step 2: Engineering new features ---")
    
    # 1. Create 'Info' column by concatenating category columns.
    for col in INFO_COLS:
        if col in df.columns:
            df[col] = df[col].replace('-', '')
    df['Info'] = df[INFO_COLS].apply(lambda row: ', '.join(filter(None, row.astype(str))), axis=1)
    print("Created 'Info' column.")

    # 2. Create 'MktCap AUM' by concatenating 'Market Cap' and 'AUM'.
    # This combines stock and ETF liquidity metrics into a single string column for now.
    # It will be converted to numeric in the next step.
    df['MktCap AUM'] = df['Market Cap'].replace('-', '') + df['AUM'].replace('-', '')
    print("Created 'MktCap AUM' column.")

    # Display the new columns for verification
    display(df[['Ticker', 'Info', 'MktCap AUM']].head(3))


--- Step 2: Engineering new features ---


Created 'Info' column.
Created 'MktCap AUM' column.


,Ticker,Info,MktCap AUM
0,SOLV,"Healthcare, Medical Instruments & Supplies",12.91B
1,REG,"Real Estate, REIT - Retail",12.88B
2,PAA,"Energy, Oil & Gas Midstream",12.84B


### Step 3: Data Type Conversion

This multi-part step cleans and converts all string-based numeric and percentage columns into proper numeric types.

#### Part A: Convert Abbreviated Currency Columns to Millions

In [4]:
def convert_to_millions(value: str) -> float:
    """Converts a string with a T/B/M/K suffix to a numeric value in millions."""
    if pd.isna(value):
        return np.nan
    
    value_str = str(value).strip().upper()
    if not value_str:
        return np.nan

    multipliers = {'T': 1_000_000, 'B': 1_000, 'M': 1, 'K': 0.001}
    suffix = value_str[-1]
    
    if suffix in multipliers:
        number_part = value_str[:-1]
        try:
            return float(number_part) * multipliers[suffix]
        except (ValueError, TypeError):
            return np.nan
    return np.nan

if df is not None:
    print("\n--- Step 3a: Converting currency columns to millions ---")
    new_names = {}
    for col in CURRENCY_COLS:
        if col in df.columns:
            df[col] = df[col].apply(convert_to_millions)
            new_names[col] = f"{col}, M"
    
    df.rename(columns=new_names, inplace=True)
    print(f"Converted and renamed {len(new_names)} columns.")
    display(df[[name for name in new_names.values() if name in df.columns]].head(3))


--- Step 3a: Converting currency columns to millions ---
Converted and renamed 12 columns.


,"Market Cap, M","AUM, M","Sales, M","Income, M","Outstanding, M","Float, M","Short Interest, M","Avg Volume, M","Flows 1M, M","Flows 3M, M","Flows YTD, M","MktCap AUM, M"
0,12910.0,NaN,8310.0,378.00,173.01,138.41,4.20,1.17,NaN,NaN,NaN,12910.0
1,12880.0,NaN,1470.0,386.55,181.05,180.20,6.21,1.13,NaN,NaN,NaN,12880.0
2,12840.0,NaN,50170.0,653.00,703.78,457.18,10.87,3.68,NaN,NaN,NaN,12840.0


#### Part B: Convert Percentage Columns to Numeric

In [5]:
if df is not None:
    print("\n--- Step 3b: Converting percentage columns ---")
    percent_cols = [
        col for col in df.columns if df[col].dtype == 'object' and df[col].str.endswith('%', na=False).any()
    ]

    if not percent_cols:
        print("No new percentage columns found to modify.")
    else:
        print("Processing the following percentage columns:")
        for col in percent_cols:
            df[col] = pd.to_numeric(df[col].str.replace('%', ''), errors='coerce')
            new_name = f"{col} %" if '%' not in col else col
            df.rename(columns={col: new_name}, inplace=True)
            print(f"  - Converted '{col}' to numeric and renamed to '{new_name}'")


--- Step 3b: Converting percentage columns ---
Processing the following percentage columns:
  - Converted 'Dividend' to numeric and renamed to 'Dividend %'
  - Converted 'Payout Ratio' to numeric and renamed to 'Payout Ratio %'
  - Converted 'EPS This Y' to numeric and renamed to 'EPS This Y %'
  - Converted 'EPS Next Y' to numeric and renamed to 'EPS Next Y %'
  - Converted 'EPS Past 5Y' to numeric and renamed to 'EPS Past 5Y %'
  - Converted 'EPS Next 5Y' to numeric and renamed to 'EPS Next 5Y %'
  - Converted 'Sales Past 5Y' to numeric and renamed to 'Sales Past 5Y %'
  - Converted 'Sales Q/Q' to numeric and renamed to 'Sales Q/Q %'
  - Converted 'EPS Q/Q' to numeric and renamed to 'EPS Q/Q %'
  - Converted 'EPS YoY TTM' to numeric and renamed to 'EPS YoY TTM %'
  - Converted 'Sales YoY TTM' to numeric and renamed to 'Sales YoY TTM %'
  - Converted 'EPS Surprise' to numeric and renamed to 'EPS Surprise %'
  - Converted 'Revenue Surprise' to numeric and renamed to 'Revenue Surprise 

  - Converted 'Profit M' to numeric and renamed to 'Profit M %'
  - Converted 'Perf Week' to numeric and renamed to 'Perf Week %'
  - Converted 'Perf Month' to numeric and renamed to 'Perf Month %'
  - Converted 'Perf Quart' to numeric and renamed to 'Perf Quart %'
  - Converted 'Perf Half' to numeric and renamed to 'Perf Half %'
  - Converted 'Perf Year' to numeric and renamed to 'Perf Year %'
  - Converted 'Perf YTD' to numeric and renamed to 'Perf YTD %'
  - Converted 'Volatility W' to numeric and renamed to 'Volatility W %'
  - Converted 'Volatility M' to numeric and renamed to 'Volatility M %'
  - Converted 'SMA20' to numeric and renamed to 'SMA20 %'
  - Converted 'SMA50' to numeric and renamed to 'SMA50 %'
  - Converted 'SMA200' to numeric and renamed to 'SMA200 %'
  - Converted '50D High' to numeric and renamed to '50D High %'
  - Converted '50D Low' to numeric and renamed to '50D Low %'
  - Converted '52W High' to numeric and renamed to '52W High %'
  - Converted '52W Low' to n

#### Part C: Convert Other String-Based Numeric Columns

In [6]:
if df is not None:
    print("\n--- Step 3c: Converting other numeric string columns ---")
    converted_count = 0
    for col in OTHER_NUMERIC_COLS:
        if col in df.columns and df[col].dtype == 'object':
            df[col] = pd.to_numeric(df[col].str.replace(',', '', regex=False), errors='coerce')
            converted_count += 1
            
    print(f"Converted {converted_count} additional columns to numeric type.")
    print("\nData types after all conversions:")
    df.info()


--- Step 3c: Converting other numeric string columns ---
Converted 32 additional columns to numeric type.

Data types after all conversions:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1560 entries, 0 to 1559
Columns: 113 entries, No. to MktCap AUM, M
dtypes: float64(94), int64(2), object(17)
memory usage: 1.3+ MB


### Step 4: Final Processing - Sort, Index, and Rank

Sort the DataFrame by the unified liquidity metric, set the `Ticker` as the index, and add a final `Rank`.

In [7]:
if df is not None:
    print("\n--- Step 4: Finalizing DataFrame ---")
    
    # 1. Sort by the primary metric in descending order
    df.sort_values(by='MktCap AUM, M', ascending=False, inplace=True, na_position='last')
    print("Sorted DataFrame by 'MktCap AUM, M'.")
    
    # 2. Add a 'Rank' column based on the new sort order
    df['Rank'] = range(1, len(df) + 1)
    print("Added 'Rank' column.")
    
    # 3. Set 'Ticker' as the index
    if 'Ticker' in df.columns:
        df.set_index('Ticker', inplace=True)
        print("Set 'Ticker' as the index.")
    
    print("\nFinal DataFrame structure:")
    display(df[['Rank', 'Info', 'MktCap AUM, M']].head())


--- Step 4: Finalizing DataFrame ---
Sorted DataFrame by 'MktCap AUM, M'.
Added 'Rank' column.
Set 'Ticker' as the index.

Final DataFrame structure:


C:\Users\ping\AppData\Local\Temp\ipykernel_16336\3337454481.py:9: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df['Rank'] = range(1, len(df) + 1)


,Rank,Info,"MktCap AUM, M"
Ticker,,,
NVDA,1,"Technology, Semiconductors",3782490.0
MSFT,2,"Technology, Software - Infrastructure",3697320.0
AAPL,3,"Technology, Consumer Electronics",3002100.0
AMZN,4,"Consumer Cyclical, Internet Retail",2305020.0
GOOG,5,"Communication Services, Internet Content & Inf...",2111210.0


### Step 5: Save and Verify Cleaned Data

Save the final, cleaned DataFrame to a new Parquet file and read it back to verify integrity.

In [8]:
if df is not None:
    print("\n--- Step 5: Saving and verifying data ---")
    try:
        # Ensure destination directory exists
        DEST_PATH.parent.mkdir(parents=True, exist_ok=True)
        
        # Save the file
        df.to_parquet(DEST_PATH, engine='pyarrow', compression='zstd')
        print(f"Successfully saved cleaned data to: {DEST_PATH}")

        # Verify by loading it back
        loaded_df = pd.read_parquet(DEST_PATH, engine='pyarrow')
        print("\nVerification successful. First 20 rows of the saved file:")
        display(loaded_df.head(20))
        
    except Exception as e:
        print(f"An error occurred during save or verification: {e}")


--- Step 5: Saving and verifying data ---
Successfully saved cleaned data to: c:\Users\ping\Files_win10\python\py311\stocks_v0_works\data\2025-06-26_df_finviz_stocks_etfs.parquet

Verification successful. First 20 rows of the saved file:


,No.,Company,Index,Sector,Industry,Country,Exchange,"Market Cap, M",P/E,Fwd P/E,PEG,P/S,P/B,P/C,P/FCF,Book/sh,Cash/sh,Dividend %,Dividend TTM,Dividend Ex Date,Payout Ratio %,EPS,EPS next Q,EPS This Y %,EPS Next Y %,EPS Past 5Y %,EPS Next 5Y %,Sales Past 5Y %,Sales Q/Q %,EPS Q/Q %,EPS YoY TTM %,Sales YoY TTM %,"Sales, M","Income, M",EPS Surprise %,Revenue Surprise %,"Outstanding, M","Float, M",Float %,Insider Own %,Insider Trans %,Inst Own %,Inst Trans %,Short Float %,Short Ratio,"Short Interest, M",ROA %,ROE %,ROIC %,Curr R,Quick R,LTDebt/Eq,Debt/Eq,Gross M %,Oper M %,Profit M %,Perf Week %,Perf Month %,Perf Quart %,Perf Half %,Perf Year %,Perf YTD %,Beta,ATR,Volatility W %,Volatility M %,SMA20 %,SMA50 %,SMA200 %,50D High %,50D Low %,52W High %,52W Low %,52W Range,All-Time High %,All-Time Low %,RSI,Earnings,IPO Date,Optionable,Shortable,Employees,Change from Open %,Gap %,Recom,"Avg Volume, M",Rel Volume,Volume,Target Price,Prev Close,Open,High,Low,Price,Change %,Single Category,Asset Type,Expense %,Holdings,"AUM, M","Flows 1M, M",Flows% 1M,"Flows 3M, M",Flows% 3M,"Flows YTD, M",Flows% YTD,Return% 1Y,Return% 3Y,Return% 5Y,Tags,Info,"MktCap AUM, M",Rank
Ticker,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
NVDA,1,NVIDIA Corp,"DJIA, NDX, S&P 500",Technology,Semiconductors,USA,NASD,3782490.0,49.93,27.12,1.69,25.47,45.09,70.45,52.49,3.44,2.20,0.03,0.04,6/11/2025,1.16,3.10,1.00,44.35,32.42,91.83,29.57,64.24,69.18,27.60,81.36,86.17,148510.0,76770.0,9.89,1.68,24390.00,23400.00,95.97,4.08,-0.36,66.39,0.57,0.88,0.82,206.80,75.89,115.46,81.82,3.39,2.96,0.12,0.12,70.11,58.03,51.69,6.56,14.41,36.27,15.09,31.25,15.44,2.13,3.92,2.27,2.36,7.93,20.50,20.22,0.37,63.11,0.37,78.97,86.62 - 154.45,0.37,464959.99,74.45,May 28/a,1/22/1999,Yes,Yes,36000.0,-0.63,1.10,1.37,250.77000,0.78,196761441,173.58,154.31,156.00,156.71,154.00,155.02,0.46,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-,"Technology, Semiconductors",3782490.0,1
MSFT,2,Microsoft Corporation,"DJIA, NDX, S&P 500",Technology,Software - Infrastructure,USA,NASD,3697320.0,38.44,32.83,2.65,13.69,11.49,46.44,53.30,43.30,10.71,0.66,2.41,8/21/2025,25.42,12.94,3.37,13.50,13.12,18.45,14.53,14.33,13.27,17.88,12.10,14.13,270010.0,96640.0,7.38,2.38,7430.00,7320.00,98.50,1.48,-0.12,73.61,0.68,0.70,2.23,51.17,18.46,33.61,23.24,1.37,1.36,0.29,0.33,69.07,45.23,35.79,3.58,7.98,27.56,13.94,11.12,18.02,1.03,6.99,1.55,1.25,4.76,12.17,17.72,0.59,39.86,0.59,44.28,344.79 - 494.56,0.59,624220.41,79.42,Apr 30/a,3/13/1986,Yes,Yes,228000.0,0.92,0.13,1.31,22.91000,0.94,21475419,519.16,492.27,492.93,498.04,492.81,497.45,1.05,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-,"Technology, Software - Infrastructure",3697320.0,2
AAPL,3,Apple Inc,"DJIA, NDX, S&P 500",Technology,Consumer Electronics,USA,NASD,3002100.0,31.37,25.87,4.09,7.50,44.95,61.90,30.48,4.47,3.25,0.51,1.01,5/12/2025,16.11,6.41,1.41,6.21,8.39,15.41,7.67,8.51,5.08,7.68,-0.36,4.91,400370.0,97290.0,1.39,0.86,14940.00,14920.00,99.88,0.10,-1.28,63.81,-0.18,0.67,1.63,100.23,29.10,138.02,66.93,0.82,0.78,1.18,1.47,46.63,31.81,24.30,2.25,0.39,-9.27,-21.02,-3.43,-19.73,1.21,4.26,1.77,1.89,0.31,-0.81,-10.04,-6.32,5.89,-22.72,18.79,169.21 - 260.10,-22.72,315858.21,49.62,May 01/a,12/12/1980,Yes,Yes,164000.0,-0.12,-0.15,2.04,61.34000,0.82,50495251,228.01,201.56,201.25,202.64,199.46,201.00,-0.28,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-,"Technology, Consumer Electronics",3002100.0,3
AMZN,4,Amazon.com Inc,"DJIA, NDX, S&P 500",Consumer Cyclical,Internet Retail,USA,NASD,2305020.0,35.41,29.86,2.06,3.54,7.53,23.46,110.77,28.82,9.25,NaN,NaN,-,0.00,6.13,1.32,12.18,17.23,36.89,17.22,17.86,8.62,62.33,71.88,10.08,650310.0,65940.0,16.38,0.33,10610.00,9490.00,89.45,10.58,-0.02,64.43,0.38,0.65,1.27,61.84,11.23,25.24,15.02,1.05,0.84,0.44,0.49,49.16,11.15,10.14,2.16,5.39,7.95,-3.47,17.00,-1.03,1.33,5.10,2.28,2.06,2.70,8.52,6.07,-0.59,31.36,-10.47,43.21,151.61 - 242.52,-10.47,330749.53,62.03,May 01/a,5